In [ ]:
# импорт библиотек

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pyarrow.parquet as pq
import gc
import seaborn as sns
from tqdm.notebook import tqdm
from catboost import CatBoostClassifier
from catboost.utils import eval_metric
from catboost import cv
from catboost import Pool
from sklearn.model_selection import train_test_split
from random import sample
%matplotlib inline
plt.style.use('seaborn')

### Загрузка данных

In [ ]:
# загрузка данных y_train

y_train = pq.ParquetFile('C:\\Hackathon\\train\\y_train.parquet')
y_train = y_train.read_row_group(0)

df = []
for col in y_train.column_names:
    if col == 'DT':
        df.append(y_train[col].to_pandas()) 
    else:
        df.append(y_train[col].to_pandas().astype('int8'))

target_df = pd.concat(df, axis=1)

del y_train
del df
del col
gc.collect()

In [ ]:
target_df.info()

In [ ]:
# загрузка данных X_train

X_train = pq.ParquetFile('C:\\Hackathon\\train\\X_train.parquet')
X_train = X_train.read_row_group(0)

df = []
for col in X_train.column_names:
    if col == 'DT':
        df.append(X_train[col].to_pandas()) 
    else:
        df.append(X_train[col].to_pandas().astype("float32"))

train_df = pd.concat(df, axis=1)

del X_train
del df
del col
gc.collect()

In [ ]:
train_df.info()

In [45]:
# Установим время в качестве индекса
target_data = target_df.copy().set_index('DT')
train_data = train_df.copy().set_index('DT')

# Возьмем отсчеты каждые 30 секунд
target_data = target_data.iloc[::3,::]
train_data = train_data.iloc[::3,::]

In [46]:
print(target_data.shape)
print(train_data.shape)

(3111678, 175)
(3111678, 96)


### Строим классификацию по принципу OneVsAll 

Для этого сформируем два датасета и обучим два классификатора 

#### Формирование датасета для классификации М3 ошибки

In [47]:
def bar_plots_target (target_df):

    """Функция построения столючатой диаграммы распределения видов по типам неисправностей

    Args:
        target_df (DataFrame): таргет датафрейм
    """
    try:
        defect_name_list = list(target_df.columns) # Список определяемых дефектов
    
        # Построение диаграмм
        fig, ax = plt.subplots(ncols=3, nrows=round(len(defect_name_list)/3), figsize=(25,len(defect_name_list)))
        for i,j in zip(range(round(len(defect_name_list)/3)), range(0,len(defect_name_list),3)):
            bar_data = target_df.iloc[:,j].value_counts().reset_index()
            sns.barplot(data=bar_data, x=bar_data['index'], y=bar_data.iloc[:,1], ax=ax[i][0]).set(xlabel='defect_type 1 - M1, 2 - M3', ylabel='count', title=defect_name_list[j])
            del bar_data

            bar_data = target_df.iloc[:,j+1].value_counts().reset_index()
            sns.barplot(data=bar_data, x=bar_data['index'], y=bar_data.iloc[:,1], ax=ax[i][1]).set(xlabel='defect_type 1 - M1, 2 - M3', ylabel='count', title=defect_name_list[j+1])
            del bar_data

            bar_data = target_df.iloc[:,j+2].value_counts().reset_index()
            sns.barplot(data=bar_data, x=bar_data['index'], y=bar_data.iloc[:,1], ax=ax[i][2]).set(xlabel='defect_type 1 - M1, 2 - M3', ylabel='count', title=defect_name_list[j+2])
            del bar_data

        plt.tight_layout()
        plt.show()
    except IndexError: 
        print('Количество ячеек больше количества графиков')

In [48]:
# Заменить все 1 (М1) на 0 
target_data = target_data.replace(1, 0)

In [ ]:
# Построим столбчатые диаграммы распределения видов и типов неисправностей 
#bar_plots_target(target_data)

In [49]:
def del_col_with_only_zero (df):
    """ Удалим таргеты, состоящиет только из нулей

    Args:
        df (pd.DataFrame()): таргейт датафрейм
        
    return:
        df (pd.DataFrame()): таргет датафрейм без таргетов, состоящих только из нулей
    """
    drop_col_list = []
    for col in df.columns.tolist():
        if (sum(df[col].value_counts().index) == 0):
            drop_col_list.append(col)

    df.drop(columns = drop_col_list, axis = 1, inplace = True)
    return df

In [50]:
# Удалим в таргете некоторые столбцы 
target_data = del_col_with_only_zero(target_data)
target_data.shape

(3111678, 140)

In [51]:
def del_str_with_only_zero(df):
    """Удалим нблюдения, состоящие только из нулей

    Args:
        df (pd.DataFrame()): таргейт датафрейм
        
    return:
        df (pd.DataFrame()): таргет датафрейм без наблюдений, состоящих только из нулей
    """
    df = df.replace(0, np.nan)
    df = df.dropna(axis = 0, how = 'all')
    df = df.fillna(0)
    return df
    

In [52]:
# Удалим в таргете некоторые строки
target_data = del_str_with_only_zero(target_data)
target_data.shape

(3098880, 140)

In [53]:
def del_col_with_one_target (df):
    """ Удалим таргеты, в который ненулевой класс менее 1%

    Args:
        df (pd.DataFrame()): таргейт датафрейм
        
    return:
        df (pd.DataFrame()): таргет датафрейм без таргетов, в который ненулевой класс менее 1%
    """
    drop_col_list = []
    for col in df.columns.tolist():
        if 100 * df[col].value_counts().iloc[1] / df[col].value_counts().iloc[0] < 1:
            drop_col_list.append(col)

    df.drop(columns = drop_col_list, axis = 1, inplace = True)
    return df

In [54]:
# Удалим в таргете некоторые столбцы
target_data = del_col_with_one_target(target_data)
target_data.shape

(3098880, 111)

In [55]:
# Сбросим индексы

train_data = train_data.reset_index()
target_data = target_data.reset_index()

In [56]:
# Объединим в один датафрейм по времени
dataset = train_data.merge(target_data, how = 'inner', on = 'DT')
dataset.shape

(3098880, 208)

In [57]:
# Разедлим датасет на трейн и таргет
train = dataset.iloc[::,1:97]
target = dataset.iloc[::,97:]

del dataset
del train_data
del target_data

In [58]:
# Заменим целевые знамения для М3 с 2 на 1
target = target.replace(2, 1)

In [59]:
# Переведем таргет в category
for col in target.columns:
    target[col] = target[col].astype('category')

In [60]:
# Раздели выборки на трейн, тест и валидацию
X_train, X_validation, y_train, y_validation = train_test_split(train, target, train_size = 0.8, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(train, target, train_size = 0.8, random_state = 41)

del train
del target
del target_df
del train_df

#### Обработка трейна

Пропуски

In [61]:
# Количество пропусков в трейне
X_train.isna().sum().sum()

35177006

In [62]:
# Обработаем пропуски, заполнив их следующим или предыдущим значением
X_train = X_train.fillna(method='ffill')
X_train = X_train.fillna(method='bfill')


In [63]:
# Количество пропусков в трейне
X_train.isna().sum().sum()

0

Мультикорреляция

In [64]:
def list_multicorr_features (features_data):
    
    """Возвращает список мультикоррелированных фичей
    Args:
        features_data (DataFrame): Датафрейм
    """
    # находим корреляционную матрицу
    corr_matrix = features_data.corr().abs()

    # создаем маску для отбора признаков с корреляцией >0.95
    mask = (corr_matrix > 0.95)

    # находим список признаков с корреляцией > 0.95
    return corr_matrix.columns[mask.any()].tolist()

In [65]:
multi_corr_feature_4 = list_multicorr_features(X_train.filter(like = 'ЭКСГАУСТЕР 4'))
print(multi_corr_feature_4)

multi_corr_feature_5 = list_multicorr_features(X_train.filter(like = 'ЭКСГАУСТЕР 5'))
print(multi_corr_feature_5)

multi_corr_feature_6 = list_multicorr_features(X_train.filter(like = 'ЭКСГАУСТЕР 6'))
print(multi_corr_feature_6)

multi_corr_feature_7 = list_multicorr_features(X_train.filter(like = 'ЭКСГАУСТЕР 7'))
print(multi_corr_feature_7)

multi_corr_feature_8 = list_multicorr_features(X_train.filter(like = 'ЭКСГАУСТЕР 8'))
print(multi_corr_feature_8)

multi_corr_feature_9 = list_multicorr_features(X_train.filter(like = 'ЭКСГАУСТЕР 9'))
print(multi_corr_feature_9)


['ЭКСГАУСТЕР 4. ТОК РОТОРА 1', 'ЭКСГАУСТЕР 4. ТОК РОТОРА2', 'ЭКСГАУСТЕР 4. ТОК СТАТОРА', 'ЭКСГАУСТЕР 4. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ', 'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1', 'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4', 'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ', 'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 1', 'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 2', 'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3', 'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4', 'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.']
['ЭКСГАУСТЕР 5. ТОК РОТОРА 1', 'ЭКСГАУСТЕР 5. ТОК РОТОРА 2', 'ЭКСГАУСТЕР 5. ТОК СТАТОРА', 'ЭКСГАУСТЕР 5. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ', 'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1', 'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА Н

In [66]:
# Выберем фичи для удаления
multi_corr_feature_4 = ['ЭКСГАУСТЕР 4. ТОК РОТОРА2',
                        'ЭКСГАУСТЕР 4. ТОК СТАТОРА', 
                        'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 
                        'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 
                        'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
                        'ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 
                         'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 2', 
                         'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3', 
                         'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 
                         'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4', 
                         'ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
                        
]

multi_corr_feature_5 = ['ЭКСГАУСТЕР 5. ТОК РОТОРА 2',
                        'ЭКСГАУСТЕР 5. ТОК СТАТОРА', 
                        'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 
                        'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 
                        'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
                        'ЭКСГАУСТЕР 5. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 
                         'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 2', 
                         'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3', 
                         'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 
                         'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4', 
                         'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
                        
]

multi_corr_feature_6 = ['ЭКСГАУСТЕР 6. ТОК РОТОРА 2',
                        'ЭКСГАУСТЕР 6. ТОК СТАТОРА', 
                        'ЭКСГАУСТЕР 6. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 
                        'ЭКСГАУСТЕР 6. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 
                        'ЭКСГАУСТЕР 6. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
                        'ЭКСГАУСТЕР 6. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 
                         'ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 2', 
                         'ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 3', 
                         'ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 
                         'ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 4', 
                         'ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
                        
]

multi_corr_feature_7 = ['ЭКСГАУСТЕР 7. ТОК РОТОРА 2',
                        'ЭКСГАУСТЕР 7. ТОК СТАТОРА', 
                        'ЭКСГАУСТЕР 7. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 
                        'ЭКСГАУСТЕР 7. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 
                        'ЭКСГАУСТЕР 7. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
                        'ЭКСГАУСТЕР 7. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 
                         'ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 2', 
                         'ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 3', 
                         'ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 
                         'ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 4', 
                         'ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
                        
]

multi_corr_feature_8 = ['ЭКСГАУСТЕР 8. ТОК РОТОРА 2',
                        'ЭКСГАУСТЕР 8. ТОК СТАТОРА', 
                        'ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 
                        'ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 
                        'ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
                        'ЭКСГАУСТЕР 8. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 
                         'ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 2', 
                         'ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 3', 
                         'ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 
                         'ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 4', 
                         'ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
                        
]

multi_corr_feature_9 = ['ЭКСГАУСТЕР 9. ТОК РОТОРА 2',
                        'ЭКСГАУСТЕР 9. ТОК СТАТОРА', 
                        'ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 
                        'ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 
                        'ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
                        'ЭКСГАУСТЕР 9. ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 
                         'ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 2', 
                         'ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 3', 
                         'ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 
                         'ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 4', 
                         'ЭКСГАУСТЕР 9. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.'
                        
]

In [67]:
# Удалим из трейна
X_train.drop(columns = multi_corr_feature_4, axis = 1, inplace = True)
X_train.drop(columns = multi_corr_feature_5, axis = 1, inplace = True)
X_train.drop(columns = multi_corr_feature_6, axis = 1, inplace = True)
X_train.drop(columns = multi_corr_feature_7, axis = 1, inplace = True)
X_train.drop(columns = multi_corr_feature_8, axis = 1, inplace = True)
X_train.drop(columns = multi_corr_feature_9, axis = 1, inplace = True)

In [68]:
X_train.shape

(2479104, 30)

In [69]:
# Удалим из валидационного множества
X_validation.drop(columns = multi_corr_feature_4, axis = 1, inplace = True)
X_validation.drop(columns = multi_corr_feature_5, axis = 1, inplace = True)
X_validation.drop(columns = multi_corr_feature_6, axis = 1, inplace = True)
X_validation.drop(columns = multi_corr_feature_7, axis = 1, inplace = True)
X_validation.drop(columns = multi_corr_feature_8, axis = 1, inplace = True)
X_validation.drop(columns = multi_corr_feature_9, axis = 1, inplace = True)

In [70]:
X_validation.shape

(619776, 30)

### Построение модели

In [72]:
model_defect_M3 = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    random_seed=35,
    save_snapshot = True,
    snapshot_file = 'model_defect_2.bkp',
    snapshot_interval = 60,
    boosting_type = 'Plain',
    bootstrap_type='Bernoulli',
    subsample=0.5,
    loss_function='MultiLogloss',
    custom_metric='F1',
    rsm = 0.5,
    leaf_estimation_iterations=5,
    max_ctr_complexity=1,
    early_stopping_rounds = 10       
)




model_defect_M3.fit(
    X_train,
    y_train,
    verbose = True,
    use_best_model = True,
    eval_set = (X_validation, y_validation)
)

0:	learn: 0.5426743	test: 0.5431736	best: 0.5431736 (0)	total: 42s	remaining: 1h 9m 14s
1:	learn: 0.4379050	test: 0.4379260	best: 0.4379260 (1)	total: 1m 56s	remaining: 1h 35m 22s
2:	learn: 0.3667408	test: 0.3655197	best: 0.3655197 (2)	total: 3m 14s	remaining: 1h 44m 59s
3:	learn: 0.3139133	test: 0.3112730	best: 0.3112730 (3)	total: 4m 24s	remaining: 1h 45m 45s
4:	learn: 0.2755272	test: 0.2725033	best: 0.2725033 (4)	total: 6m 47s	remaining: 2h 8m 56s
5:	learn: 0.2479478	test: 0.2450402	best: 0.2450402 (5)	total: 8m 28s	remaining: 2h 12m 49s
6:	learn: 0.2263406	test: 0.2234025	best: 0.2234025 (6)	total: 10m 13s	remaining: 2h 15m 52s
7:	learn: 0.2078764	test: 0.2045751	best: 0.2045751 (7)	total: 11m 53s	remaining: 2h 16m 45s
8:	learn: 0.1939873	test: 0.1901748	best: 0.1901748 (8)	total: 13m 27s	remaining: 2h 16m 3s
9:	learn: 0.1834115	test: 0.1790338	best: 0.1790338 (9)	total: 15m 9s	remaining: 2h 16m 23s
10:	learn: 0.1740159	test: 0.1694553	best: 0.1694553 (10)	total: 16m 45s	remaining:

In [73]:
y_pred = model_defect_M3.predict(X_test)

In [78]:
#Сохранение модели
model_defect_M3.save_model('model_detect_defect.bin')
model_defect_M3.save_model('model_detect_defect.json', format = 'json')

In [77]:
# 'Precision'
precision_dict = {}
values = eval_metric(y_test.values, y_pred, 'Precision')
for cls, value in zip(model_defect_M3.classes_, values):
    precision_dict[cls] = value

#'Recall'
recall_dict = {}
values = eval_metric(y_test.values, y_pred, 'Recall')
for cls, value in zip(model_defect_M3.classes_, values):
    recall_dict[cls] = value

#'F1'
f1_dict = {}
values = eval_metric(y_test.values, y_pred, 'F1')
for cls, value in zip(model_defect_M3.classes_, values):
    f1_dict[cls] = value


# Посчитаем среднее значение метрик по всем таргетам
print(f'Среднее значение метрики Precision -  {np.mean(list(precision_dict.values()))}')
print(f'Среднее значение метрики Recall -  {np.mean(list(recall_dict.values()))}')
print(f'Среднее значение метрики F1 -  {np.mean(list(f1_dict.values()))}')

Среднее значение метрики Precision -  0.9802759348371249
Среднее значение метрики Recall -  0.9620876753377553
Среднее значение метрики F1 -  0.9704608966080792
